In [51]:
import warnings 
warnings.filterwarnings("ignore")

In [52]:
import pandas as pd
import os
import random
import numpy as np
import yaml
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
from datetime import datetime, timezone, timedelta
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import LabelEncoder
from wandb.lightgbm import wandb_callback, log_summary

In [53]:
#wandb_callback 수정 
from typing import TYPE_CHECKING, Callable
import wandb
from wandb.sdk.lib import telemetry as wb_telemetry

MINIMIZE_METRICS = [
    "l1",
    "l2",
    "rmse",
    "mape",
    "huber",
    "fair",
    "poisson",
    "gamma",
    "binary_logloss",
]

MAXIMIZE_METRICS = ["map", "auc", "average_precision"]

# def _define_metric(data: str, metric_name: str) -> None:
    
#     """Capture model performance at the best step.

#     instead of the last step, of training in your `wandb.summary`
#     """
#     if "loss" in str.lower(metric_name):
#         wandb.define_metric(f"{data}_{metric_name}", summary="min")
#     elif str.lower(metric_name) in MINIMIZE_METRICS:
#         wandb.define_metric(f"{data}_{metric_name}", summary="min")
#     elif str.lower(metric_name) in MAXIMIZE_METRICS:
#         wandb.define_metric(f"{data}_{metric_name}", summary="max")
        
def wandb_callback(log_params=True, define_metric=True) -> Callable:
    """Automatically integrates LightGBM with wandb.

    Arguments:
        log_params: (boolean) if True (default) logs params passed to lightgbm.train as W&B config
        define_metric: (boolean) if True (default) capture model performance at the best step, instead of the last step, of training in your `wandb.summary`

    Passing `wandb_callback` to LightGBM will:
      - log params passed to lightgbm.train as W&B config (default).
      - log evaluation metrics collected by LightGBM, such as rmse, accuracy etc to Weights & Biases
      - Capture the best metric in `wandb.summary` when `define_metric=True` (default).

    Use `log_summary` as an extension of this callback.

    Example:
        ```python
        params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            .
        }
        gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=10,
                        valid_sets=lgb_eval,
                        valid_names=('validation'),
                        callbacks=[wandb_callback()])
        ```
    """
    def _define_metric(data: str, metric_name: str) -> None:
    
        """Capture model performance at the best step.
        instead of the last step, of training in your `wandb.summary`
        """
        if "loss" in str.lower(metric_name):
            wandb.define_metric(f"{data}_{metric_name}", summary="min")
        elif str.lower(metric_name) in MINIMIZE_METRICS:
            wandb.define_metric(f"{data}_{metric_name}", summary="min")
        elif str.lower(metric_name) in MAXIMIZE_METRICS:
            wandb.define_metric(f"{data}_{metric_name}", summary="max")
            
    log_params_list: "List[bool]" = [log_params]
    define_metric_list: "List[bool]" = [define_metric]

    def _init(env: "CallbackEnv") -> None:
        with wb_telemetry.context() as tel:
            tel.feature.lightgbm_wandb_callback = True

        wandb.config.update(env.params)
        log_params_list[0] = False

        if define_metric_list[0]:
            for i in range(len(env.evaluation_result_list)):
                data_type = env.evaluation_result_list[i][0]
                metric_name = env.evaluation_result_list[i][1]
                _define_metric(data_type, metric_name)

    def _callback(env: "CallbackEnv") -> None:
        if log_params_list[0]:
            _init(env)
        # eval_results: "Dict[str, Dict[str, List[Any]]]" = {}
        # recorder = lightgbm.record_evaluation(eval_results)
        # recorder(env)
        eval_results = {x[0]:{x[1:][0]:x[1:][1:]} for x in env.evaluation_result_list}

        for validation_key in eval_results.keys():
            for key in eval_results[validation_key].keys():
                 wandb.log(
                     {validation_key + "_" + key: eval_results[validation_key][key][0]},
                     commit=False,
                 )
        for item in eval_results:
            if len(item) == 4:
                wandb.log({f"{item[0]}_{item[1]}": item[2]}, commit=False)

        # Previous log statements use commit=False. This commits them.
        wandb.log({"iteration": env.iteration}, commit=True)

    return _callback

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.


## Training

In [54]:
sweep_config_path = '/data/ephemeral/level2-dkt-recsys-06/code/boost/lgbm.yaml'

# YAML 파일 로드
with open(sweep_config_path, 'r') as file:
    sweep_config = yaml.safe_load(file)

# W&B 스위프트 설정
sweep_id = wandb.sweep(sweep=sweep_config, project="lightgbm-sweep")


wandb: WARNING Config item 'num_leaves' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_depth' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'min_data_in_leaf' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'feature_fraction' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'bagging_fraction' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'bagging_freq' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lambda_l1' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lambda_l2' was locked by 'sweep' (ignored update).


wandb: WARNING Config item 'cat_smooth' was locked by 'sweep' (ignored update).


[LightGBM] [Info] Number of positive: 3253, number of negative: 2102
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.033587 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2820
[LightGBM] [Info] Number of data points in the train set: 5355, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.607470 -> initscore=0.436688
[LightGBM] [Info] Start training from score 0.436688
Training until validation scores don't improve for 20 rounds


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Early stopping, best iteration is:
[14]	training's auc: 0.856605	valid_1's auc: 0.819895
Fold 3 VALID AUC : 0.8198953823953824 ACC : 0.7516778523489933

ACU[0.77996255 0.81100037 0.85271739 0.81989538 0.        ]
Fold 5:


fold,▁
fold_accuracy,▁
fold_auc,▁
iteration,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇███
training_auc,▁▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇██████████████████
valid_1_auc,▁▆▇▇▇▇██▇█▇▇██████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
fold,3
fold_accuracy,0.75168
fold_auc,0.8199
iteration,33


wandb: WARNING Config item 'num_leaves' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_depth' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'min_data_in_leaf' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'feature_fraction' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'bagging_fraction' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'bagging_freq' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lambda_l1' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lambda_l2' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'cat_smooth' was locked by 'sweep' (ignored update).


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 3257, number of negative: 2107
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000441 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2822
[LightGBM] [Info] Number of data points in the train set: 5364, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.607196 -> initscore=0.435541
[LightGBM] [Info] Start training from score 0.435541
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[2]	training's auc: 0.819169	valid

fold,▁
fold_accuracy,▁
fold_auc,▁
iteration,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
mean_accuracy,▁
mean_auc,▁
training_auc,▁▆▆▇▇▇▇▇▇▇████████████
valid_1_auc,▁███▇█▇██▇▇▇▇▇▇▇▇▇▇▇▇▇
fold,4
fold_accuracy,0.7973
fold_auc,0.8665


Create sweep with ID: ojwoo0ep
Sweep URL: https://wandb.ai/boostcamp6-recsys6/lightgbm-sweep/sweeps/ojwoo0ep


In [55]:
# sweep_config = '/data/ephemeral/level2-dkt-recsys-06/code/boost/lgbm.yaml'  # YAML 파일 경로
# sweep_id = wandb.sweep(sweep=sweep_config, project="lightgbm-sweep")

In [56]:

# 노트북의 이름 설정
os.environ['WANDB_NOTEBOOK_NAME'] = 'LGBM_Train.ipynb'


X = pd.read_csv('/data/ephemeral/level2-dkt-recsys-06/data/FE_Test_v4_2.csv')
test =  pd.read_csv('/data/ephemeral/level2-dkt-recsys-06/data/FE_Test_v4_2.csv')
# 유저 최근데이터만 사용
X = X.groupby('userID').tail(10)


X = X[X['answerCode'] != -1]

X.shape



(6696, 23)

In [57]:
y=X["answerCode"]
g=X["userID"]

In [58]:
feat=[ 'userID','assessmentItemID','testId','KnowledgeTag',
       'SolvingTime','CumulativeTime',
       'Month','DayOfWeek','TimeOfDay',
       'problems_cumulative','problems_last7days','problems_last30days',
       'CumulativeUserProblemAnswerRate','CumulativeProblemCount',
       'ProblemAnswerRate','TagAnswerRate','CumulativeUserTagAnswerRate','TestAnswerRate',
       'categorize_solvingTime','categorize_ProblemAnswerRate','categorize_TagAnswerRate','categorize_TestAnswerRate'
]

In [59]:
params = {}

# LabelEncoder 적용
label_encoders = {}
for column in [
    "DayOfWeek",
    "TimeOfDay",
    "categorize_ProblemAnswerRate",
    "categorize_TagAnswerRate",
    "categorize_TestAnswerRate",
]:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column])
    # 테스트 데이터에 대해서는 transform만 적용
    test[column] = le.transform(test[column])
    label_encoders[column] = le


test = test[test["userID"] != test["userID"].shift(-1)]
test = test.drop(["answerCode"], axis=1)
n_fold = 5

sfcv = StratifiedGroupKFold(n_splits=n_fold)


def train():
    HPPJ = 1
    oof_auc = np.zeros(n_fold)
    oof_acc = np.zeros(n_fold)
    test_preds = np.zeros(len(test))
    # X = X.drop(columns=['Timestamp'])
    # test = test.drop(columns=['Timestamp'])
    for i, (train_idx, val_idx) in enumerate(sfcv.split(X, y, g)):
        print(f"Fold {i+1}:")
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_valid = X.iloc[val_idx]
        X_valid = X_valid[X_valid["userID"] != X_valid["userID"].shift(-1)]
        y_valid = X_valid["answerCode"]
        # print(X_valid)
        # print(y_valid)
        # break

        lgb_train = lgb.Dataset(
            X_train[feat], y_train, categorical_feature=["KnowledgeTag"]
        )
        lgb_valid = lgb.Dataset(
            X_valid[feat], y_valid, categorical_feature=["KnowledgeTag"]
        )
        wandb.init(project=f"lightgbm-sweep")
        wandb.run.name = f"fold{i+1}sweep{str(HPPJ)}lgbm"
        current_params = {
            "objective": "binary",
            "metric": ['auc'],
            "device": "cpu",
            "num_leaves": wandb.config.num_leaves,
            "learning_rate": wandb.config.learning_rate,
            "max_depth": wandb.config.max_depth,
            "min_data_in_leaf": wandb.config.min_data_in_leaf,
            "feature_fraction": wandb.config.feature_fraction,
            "bagging_fraction": wandb.config.bagging_fraction,
            "bagging_freq": wandb.config.bagging_freq,
            "lambda_l1": wandb.config.lambda_l1,
            "lambda_l2": wandb.config.lambda_l2,
            "cat_smooth": wandb.config.cat_smooth,
        }
        model = lgb.train(
            current_params,
            lgb_train,
            valid_sets=[lgb_train, lgb_valid],
            num_boost_round=500,
            callbacks=[
                wandb_callback(log_params=True, define_metric=True),
                lgb.early_stopping(20),
            ],
            categorical_feature=["KnowledgeTag"],
        )   
        preds = model.predict(X_valid[feat])
        oof_acc[i] = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
        oof_auc[i] = roc_auc_score(y_valid, preds)

        wandb.log({"fold": i, "fold_accuracy": oof_acc[i], "fold_auc": oof_auc[i]})
        # print(f"predict.test{model.predict(test)}")
        test_preds += model.predict(test) / n_fold
        # print(f"test_preds{test_preds}")
        print(f"Fold {i} VALID AUC : {oof_auc[i]} ACC : {oof_acc[i]}\n")
        print(f"ACU{oof_auc}")
    # 모든 폴드의 평균 AUC 계산
    mean_auc = np.mean(oof_auc)
    mean_acc = np.mean(oof_acc)
    print(f'mean_auc{mean_auc} mean_acc{mean_acc}')
    wandb.log({"mean_auc": mean_auc, "mean_accuracy": mean_acc})

    output_dir = "output/"
    write_path = os.path.join(
        output_dir,
        datetime.now(timezone(timedelta(hours=9))).strftime("%Y-%m-%d %H:%M:%S")
        + "sweep"
        + str(HPPJ)
        + " lgbm submission.csv",
    )
    HPPJ += 1
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    with open(write_path, "w", encoding="utf8") as w:
        print("writing prediction : {}".format(write_path))
        w.write("id,prediction\n")
        for id, p in enumerate(test_preds):
            w.write("{},{}\n".format(id, p))

In [60]:

wandb.agent(sweep_id, train)

wandb: Agent Starting Run: vbrerufk with config:
wandb: 	bagging_fraction: 0.6024589332457962
wandb: 	bagging_freq: 8
wandb: 	cat_smooth: 77
wandb: 	feature_fraction: 0.8690817518815048
wandb: 	lambda_l1: 3.5103871889805616
wandb: 	lambda_l2: 0.6572678176557245
wandb: 	learning_rate: 0.0368087609710976
wandb: 	max_depth: 11
wandb: 	min_data_in_leaf: 64
wandb: 	num_leaves: 36


Fold 1:


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'num_leaves' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_depth' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'min_data_in_leaf' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'feature_fraction' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'bagging_fraction' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'bagging_freq' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lambda_l1' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lambda_l2' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'cat_smooth' was locked by 'sweep' (ignored update).
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 3254, number of negative: 2101
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000468 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2820
[LightGBM] [Info] Number of data points in the train set: 5355, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.607656 -> initscore=0.437472
[LightGBM] [Info] Start training from score 0.437472
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

...

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Early stopping, best iteration is:
[1]	training's auc: 0.82662	valid_1's auc: 0.817731
Fold 3 VALID AUC : 0.8177308802308803 ACC : 0.7114093959731543

ACU[0.80131086 0.82908822 0.82155797 0.81773088 0.        ]
Fold 5:


fold,▁
fold_accuracy,▁
fold_auc,▁
iteration,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
training_auc,▁▂▃▄▄▅▅▅▅▆▆▆▇▇▇▇▇▇███
valid_1_auc,█▂▄▃▃▅▅▆▆▅▅▃▄▄▃▄▃▁▁▂▃
fold,3
fold_accuracy,0.71141
fold_auc,0.81773
iteration,20


wandb: WARNING Config item 'num_leaves' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_depth' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'min_data_in_leaf' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'feature_fraction' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'bagging_fraction' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'bagging_freq' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lambda_l1' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lambda_l2' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'cat_smooth' was locked by 'sweep' (ignored update).


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 3257, number of negative: 2107
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000503 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2822
[LightGBM] [Info] Number of data points in the train set: 5364, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.607196 -> initscore=0.435541
[LightGBM] [Info] Start training from score 0.435541
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[2]	training's auc: 0.846675	valid_1's auc: 0.876267
Fold 4 VALID AUC : 0.8762668140777593 ACC : 0.777027027027027

ACU[0.80131086 0.82908822 0.82155797 0.81773088 0.87626681]
mean_auc

In [ ]:
np.mean(oof_auc), np.mean(oof_acc)

NameError: name 'oof_auc' is not defined

In [ ]:
output_dir = 'output/'
write_path = os.path.join(output_dir, datetime.now(timezone(timedelta(hours=9))).strftime("%Y-%m-%d %H:%M:%S")+" lgbm submission.csv")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
with open(write_path, 'w', encoding='utf8') as w:
    print("writing prediction : {}".format(write_path))
    w.write("id,prediction\n")
    for id, p in enumerate(test_preds):
        w.write('{},{}\n'.format(id,p))

writing prediction : output/2024-01-21 08:43:49 lgbm submission.csv


NameError: name 'test_preds' is not defined